<a href="https://colab.research.google.com/github/SahelKherad/3-story-benchmark-transformer/blob/main/1404_09_01(1_sensor_3_state).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import scipy.io
from scipy.io.matlab._mio5_params import mat_struct
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!ls "/content/drive/My Drive/ASCE"

In [ ]:
def load_mat(path):
    mat = scipy.io.loadmat(path, squeeze_me=True, struct_as_record=False)
    raw = mat.get('dasy')
    if isinstance(raw, mat_struct):
        channels = []
        for f in raw._fieldnames:
            arr = getattr(raw, f)
            channels.append(np.asarray(arr).reshape(-1))
        data = np.stack(channels, axis=1)  # (n_samples, n_channels)
    else:
        raise ValueError("Unexpected MAT structure. 'dasy' not a struct.")
    return data  # float64 by default

base_dir = '/content/drive/MyDrive/ASCE/'
state_files = {
    1: 'shm01a.mat', 2: 'shm02a.mat', 3: 'shm03a.mat',
    4: 'shm04a.mat', 5: 'shm05a.mat', 6: 'shm06a.mat',
    7: 'shm07a.mat', 8: 'shm08a.mat', 9: 'shm09a.mat',
}
# label_to_class_id: 1..9 -> 0..8
label_to_cid = {lab: i for i, lab in enumerate(sorted(state_files.keys()))}

scenario_data = {}   # cid -> np.ndarray [N, C]
for lab, fname in state_files.items():
    x = load_mat(os.path.join(base_dir, fname)).astype(np.float32)
    scenario_data[label_to_cid[lab]] = x
print(scenario_data[0][:,:].shape)
print(scenario_data[0][:,4].shape)
print(scenario_data[4][:,4].shape)
    # print(fname, x.shape)

In [ ]:
win = 512
stride = 128

In [ ]:
healthy_data = scenario_data[0][:,4]
damaged_data_1 = scenario_data[7][:,4]
damaged_data_2 = scenario_data[8][:,4]
print(healthy_data.shape)
print(damaged_data_1.shape)
print(damaged_data_2.shape)

In [ ]:
n_train = 0.6
n_val = 0.2

m_0 = int(n_train * len(healthy_data))
l_0 = int(n_val * len(healthy_data))

m_1 = int(n_train * len(damaged_data_1))
l_1 = int(n_val * len(damaged_data_1))

m_2 = int(n_train * len(damaged_data_2))
l_2 = int(n_val * len(damaged_data_2))

X_train_0 = healthy_data[0:m_0]
X_val_0 = healthy_data[m_0 :m_0+l_0]
X_test_0 = healthy_data[m_0+l_0:]

X_train_1 = damaged_data_1[0:m_1]
X_val_1 = damaged_data_1[m_1 :m_1+l_1]
X_test_1 = damaged_data_1[m_1+l_1:]

X_train_2 = damaged_data_2[0:m_2]
X_val_2 = damaged_data_2[m_2 :m_2+l_2]
X_test_2 = damaged_data_2[m_2+l_2:]

In [ ]:
print('x_train_o.shape:', X_train_0.shape)
print('x_val_o.shape:', X_val_0.shape)
print('x_test_o.shape:', X_test_0.shape)
print('x_train_1.shape:', X_train_1.shape)
print('x_train_2.shape:', X_train_2.shape)

In [ ]:
con = np.concatenate([X_train_0,X_train_1, X_train_2])
print(con.shape)
mu = con.mean(axis=(0) , keepdims = True)
print('mu:',mu)
std = con.std(axis=(0), keepdims=True) + 1e-6

In [ ]:
X_train_0 = (X_train_0 - mu) / std
X_val_0   = (X_val_0   - mu) / std
X_test_0  = (X_test_0  - mu) / std

X_train_1 = (X_train_1 - mu) / std
X_val_1   = (X_val_1   - mu) / std
X_test_1  = (X_test_1  - mu) / std

X_train_2 = (X_train_2 - mu) / std
X_val_2   = (X_val_2   - mu) / std
X_test_2  = (X_test_2  - mu) / std

In [ ]:
print(X_train_0.shape)
print(X_val_0.shape)
print(X_test_0.shape)

In [ ]:

def make_windows_with_stride(data, window_length, stride):

    num_frames = int((len(data) - window_length) // (stride + 1e-8) + 1)
    print(num_frames)

    if num_frames <= 0:
        # اگر طول داده برای یک پنجره هم کافی نبود
        return np.empty((0, window_length), dtype=data.dtype)

    # 2. ایجاد اندیس‌های شروع برای هر پنجره
    # شروع: 0، پایان: اندیس لازم برای آخرین پنجره کامل
    start_indices = np.arange(num_frames) * stride

    # 3. ایجاد یک ماتریس اندیس برای استخراج داده ها
    # این کار از حلقه زدن کندتر جلوگیری می کند و کل فرآیند را برداری (Vectorized) می کند
    window_indices = start_indices[:, None] + np.arange(window_length)

    # 4. استخراج داده ها با استفاده از اندیس‌های محاسبه شده
    windows = data[window_indices]

    return windows

In [ ]:
win = 512
stride = 512

In [ ]:
# 2) Make windows per class
X_train_0 = make_windows_with_stride(X_train_0, win, stride)
X_val_0   = make_windows_with_stride(X_val_0, win, stride)
X_test_0  = make_windows_with_stride(X_test_0, win, stride)

y_0 = np.zeros((X_train_0.shape[0]+X_val_0.shape[0]+X_test_0.shape[0]), dtype=np.int64)

print('Train_0 = \n')
print(X_train_0.shape)
print(X_val_0.shape)
print(X_test_0.shape)
print(y_0.shape)

X_train_1 = make_windows_with_stride(X_train_1, win, stride)
X_val_1 = make_windows_with_stride(X_val_1, win, stride)
X_test_1 = make_windows_with_stride(X_test_1, win, stride)

y_1 = np.ones((X_train_1.shape[0]+X_val_1.shape[0]+X_test_1.shape[0]), dtype=np.int64)

print('\n Train_1 = \n')
print(X_train_1.shape)
print(X_val_1.shape)
print(X_test_1.shape)
print(y_1.shape)

X_train_2 = make_windows_with_stride(X_train_2, win, stride)
X_val_2 = make_windows_with_stride(X_val_2, win, stride)
X_test_2 = make_windows_with_stride(X_test_2, win, stride)

y_2 = 2 * np.ones((X_train_2.shape[0]+X_val_2.shape[0]+X_test_2.shape[0]), dtype=np.int64)

print('\n Train_1 = \n')
print(X_train_2.shape)
print(X_val_2.shape)
print(X_test_2.shape)
print(y_2.shape)

In [ ]:
# def make_windows(data, window_length, drop_last : bool = True):
#     # Calculate how many full frames can be created
#     num_frames = len(data) // window_length
#     # print(num_frames)
#     # Trim the data to be a perfect multiple of the frame length
#     trimmed_data = data[:num_frames * window_length]

#     # Reshape the data into a 2D array of frames
#     windows = trimmed_data.reshape((num_frames, window_length))
#     # return trimmed_data.shape , frames.shape
#     return windows

In [ ]:
# win = 300
# 2) Make windows per class
# X_train_0 = make_windows(X_train_0, win, drop_last=True)
# X_val_0 = make_windows(X_val_0, win, drop_last=True)
# X_test_0 = make_windows(X_test_0, win, drop_last=True)

# y_0 = np.zeros((X_train_0.shape[0]+X_val_0.shape[0]+X_test_0.shape[0]), dtype=np.int64)

# print('Train_0 = \n')
# print(X_train_0.shape)
# print(X_val_0.shape)
# print(X_test_0.shape)
# print(y_0.shape)

# X_train_1 = make_windows(X_train_1, win, drop_last=True)
# X_val_1 = make_windows(X_val_1, win, drop_last=True)
# X_test_1 = make_windows(X_test_1, win, drop_last=True)

# y_1 = np.ones((X_train_1.shape[0]+X_val_1.shape[0]+X_test_1.shape[0]), dtype=np.int64)

# print('\n Train_1 = \n')
# print(X_train_1.shape)
# print(X_val_1.shape)
# print(X_test_1.shape)
# print(y_1.shape)

In [ ]:
X_train = np.vstack([X_train_0, X_train_1, X_train_2])[:, :, None]  # [N, 5, 1]
print(X_train.shape)

y_train = np.concatenate([y_0[0:int(len(X_train_0))], y_1[0:int(len(X_train_1))], y_2[0:int(len(X_train_2))]])             # [N]
print(y_train)
print(y_train.shape)

In [ ]:
rng = np.random.default_rng(42)
perm = rng.permutation(len(X_train))

X_train = X_train[perm]
y_train = y_train[perm]
print(y_train)

In [ ]:
X_val = np.vstack([X_val_0, X_val_1, X_val_2], )[:, :, None]  # [N, 5, 1]
print(X_val.shape)

y_val = np.concatenate([y_0[int(len(X_train_0)):int(len(X_train_0) + len(X_val_0))], y_1[int(len(X_train_1)):int(len(X_train_1) + len(X_val_1))], y_2[int(len(X_train_2)):int(len(X_train_2) + len(X_val_2))]])             # [N]
print(y_val)
print(y_val.shape)

In [ ]:
X_test =  np.vstack([X_test_0, X_test_1, X_test_2])[:, :, None]  # [N, 5, 1]
print(X_test.shape)

y_test = np.concatenate([y_0[int(len(X_train_0) + len(X_val_0)):], y_1[int(len(X_train_1) + len(X_val_1)):], y_2[int(len(X_train_2) + len(X_val_2)):]], axis=0)             # [N]
print(y_test)
print(y_test.shape)

In [ ]:
# 5) Torch tensors
X_train = torch.from_numpy(X_train).float()  # float32
y_train = torch.from_numpy(y_train).long()  # int64
X_val   = torch.from_numpy(X_val).float()
y_val   = torch.from_numpy(y_val).long()
X_test  = torch.from_numpy(X_test).float()
y_test  = torch.from_numpy(y_test).long()

In [ ]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

In [ ]:
class SeqDataset(Dataset):
    def __init__(self, X: torch.Tensor, y: torch.Tensor):
        self.X = X  # [N, seq_len, 1]
        self.y = y  # [N]
    def __len__(self): return self.y.shape[0]
    def __getitem__(self, i): return self.X[i], self.y[i]

train_ds = SeqDataset(X_train, y_train)
# print(X_train.shape)
# print(train_ds[0])
# print(train_ds[1])
# print(train_ds[2])
val_ds   = SeqDataset(X_val,   y_val)
test_ds  = SeqDataset(X_test,  y_test)

In [ ]:
# 6) DataLoaders: shuffle only the training set
batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

In [ ]:
with torch.no_grad():
    ys_tr = torch.cat([yb.view(-1) for _, yb in train_loader], dim=0).cpu()
    ys_va = torch.cat([yb.view(-1) for _, yb in val_loader], dim=0).cpu()

print("train label set/counts:", ys_tr.unique(return_counts=True))
print("val   label set/counts:", ys_va.unique(return_counts=True))
print("train min/max:", ys_tr.min().item(), ys_tr.max().item())
print("val   min/max:", ys_va.min().item(), ys_va.max().item())

In [ ]:
# 7) Example: adapt a batch for a Transformer [seq_len, batch, d_model]
xb, yb = next(iter(train_loader))   # xb: [B, 5, 1]

In [ ]:
class LearnablePositionalEncoding(nn.Module):
    """
    Learnable positional encoding added to token embeddings.
    Shape stored: [1, max_len, d_model]
    """
    def __init__(self, d_model: int, max_len: int):
        super().__init__()
        self.pe = nn.Parameter(torch.zeros(1, max_len, d_model))
        print(self.pe)
        nn.init.trunc_normal_(self.pe, std=0.02)

    def forward(self, x):
        # x: [B, T, d_model]
        T = x.size(1)
        return x + self.pe[:, :T, :]

In [ ]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, d_model=256, nhead=4, num_layers=3, d_ff=256, seq_len= X_train.shape[1], num_classes=3, dropout=0.1):
        super().__init__()
        self.inp = nn.Linear(1, d_model)                            # project feature_dim=1 -> d_model
        self.pos = LearnablePositionalEncoding(d_model, seq_len)    # learnable PE
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead,
            dim_feedforward=d_ff, dropout=dropout,
            batch_first=True                                        # so we keep [B, T, d]
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, num_classes)

    def forward(self, x):
        # x: [B, T, 1]
        x = self.inp(x)                  # [B, T, d]
        x = self.pos(x)                  # add learnable PE
        x = self.encoder(x)              # [B, T, d]
        x = self.norm(x)
        x = x.mean(dim=1)                # mean pool over time

        logits = self.head(x)            # [B, C]
        return logits

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seq_len = X_train.shape[1]
num_classes = 2
model = TimeSeriesTransformer(
    d_model=64,
    nhead=2,
    num_layers=4,
    d_ff=128,
    seq_len=seq_len ,
    num_classes=3,
    dropout=0.1).to(device)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device) # Move the model to the selected device

In [ ]:
# Optional class weights for CE or focal alpha; compute from training labels
class_counts = torch.bincount(y_train)
class_weights = (class_counts.sum() / (class_counts.float().clamp(min=1)))  # inverse freq
class_weights = class_weights / class_weights.sum() * num_classes           # normalize
class_weights = class_weights.to(device)

In [ ]:
# from torch.optim.lr_scheduler import CosineAnnealingLR
# # optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0)
# # loss_fn = nn.CrossEntropyLoss()
# from torch.optim.lr_scheduler import CosineAnnealingLR
# scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

loss_fn = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 20
tr_losses, tr_accs, val_losses, val_accs = [], [], [], []

for epoch in range(num_epochs):
     model.train()
     total_loss, total_samples, correct_predictions = 0, 0, 0
     for xb, yb in train_loader:
        xb , yb = xb.to(device) , yb.to(device).long()
        optimizer.zero_grad()
        output = model(xb)
        # print('output',output)
        # print('max',torch.max(output, 1))
        # print('yb',yb)

        loss = loss_fn(output , yb)
        # print('loss',loss)
        # optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
        # print(loss.item())
        # print(xb.size(0))
        # print('total:',total_loss)

        # Calculate accuracy
        pred = output.argmax(dim=1) # the index of the maximum value for each column
        # print('p',predicted)
        # print('real',yb)
        total_samples += xb.size(0)
        # print(total_samples)
        correct_predictions += (pred == yb).sum().item()
        # print(correct_predictions)

     epoch_loss = total_loss / total_samples
     epoch_accuracy = correct_predictions / total_samples
     tr_losses.append(epoch_loss)
     tr_accs.append(epoch_accuracy)

    #  print(f"Epoch {epoch+1}/{num_epochs}: " f"Train Loss: {epoch_loss:.4f}, "f"Train Accuracy: {epoch_accuracy:.4f}")

     model.eval()
     val_correct_predictions , val_total_samples , val_total_loss = 0 , 0 , 0

     with torch.no_grad():
          for xb, yb in val_loader:
              xb, yb = xb.to(device), yb.to(device).long()
              output = model(xb)
              val_loss = loss_fn(output,yb)
              val_total_loss += val_loss.item() * xb.size(0)
              pred = output.argmax(dim=1)
              val_total_samples += xb.size(0)
              val_correct_predictions += (pred == yb).sum().item()

     val_epoch_loss = val_total_loss / val_total_samples
     val_epoch_accuracy = val_correct_predictions / val_total_samples
    #  print(    f"Validation Loss: {val_epoch_loss:.4f}, "f"Validation Accuracy: {val_epoch_accuracy:.4f}\n")
     val_losses.append(val_epoch_loss)
     val_accs.append(val_epoch_accuracy)
     # 3) Step the scheduler once per epoch, after validation
    #  scheduler.step()

     # Optional: print current lr to confirm schedule
     current_lr = optimizer.param_groups[0]["lr"]
     print(f"Epoch {epoch+1}/{num_epochs}: " f"Train Loss: {epoch_loss:.4f}, "f"Train Accuracy: {epoch_accuracy:.4f} | "f"Validation Loss: {val_epoch_loss:.4f}, "f"Validation Accuracy: {val_epoch_accuracy:.4f}\n")




In [ ]:
import torch.nn.functional as F
val_probs = []
val_true  = []
with torch.no_grad():
    for xb, yb in val_loader:
        out = model(xb.to(device))
        val_probs.append(F.softmax(out, dim=1).max(1).values.cpu())
        val_true.append(yb)
val_probs = torch.cat(val_probs)
print('Val avg max-softmax:', val_probs.mean().item())

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
all_pred, all_true = [], []
with torch.no_grad():
    for xb, yb in val_loader:
        out = model(xb.to(device))
        all_pred.append(out.argmax(1).cpu().numpy())
        all_true.append(yb.numpy())
cm = confusion_matrix(np.concatenate(all_true), np.concatenate(all_pred))
print(cm)

In [ ]:
epoch = np.arange(0,num_epochs , 1)
import matplotlib.pyplot as plt

plt.plot(epoch, tr_losses , label="Train_loss", c='b')
plt.plot(epoch, val_losses, label="val_losss", c='r')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
import matplotlib.pyplot as plt

y_true, y_pred, y_score = [], [], []
model.eval()

with torch.no_grad():
    for xb, yb in val_loader:
        xb, yb = xb.to(device), yb.to(device).long()   # targets should be int for CE
        logits = model(xb)                             # [B,2]

        # 1) Probability of class 1
        probs  = torch.softmax(logits, dim=1)          # [B,2]
        score1 = probs[:, 1]                           # [B]

        # 2) Predicted class index
        preds  = logits.argmax(dim=1)                  # [B]

        # Save results
        y_true.extend(yb.cpu().tolist())
        y_pred.extend(preds.cpu().tolist())
        y_score.extend(score1.cpu().tolist())

# 1) Confusion Matrix
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, cmap="Blues", values_format="d")
plt.title("Confusion Matrix (Validation)")
plt.show()

# 2) ROC
RocCurveDisplay.from_predictions(y_true, y_score)
plt.title("ROC Curve (Validation)")
plt.show()

# 3) Precision–Recall
PrecisionRecallDisplay.from_predictions(y_true, y_score)
plt.title("Precision–Recall Curve (Validation)")
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torch

model.eval()
all_embeds, all_labels = [], []

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        # Pass through encoder only (skip classifier head)
        embeds = model.encoder(model.inp(xb) + model.pos(model.inp(xb)))
        embeds = embeds.mean(dim=1)                # [B, d_model]

        all_embeds.append(embeds.cpu())
        all_labels.append(yb.cpu())

# Concatenate all batches
all_embeds = torch.cat(all_embeds, dim=0).numpy()
all_labels = torch.cat(all_labels, dim=0).numpy()

# Run t-SNE
X_2d = TSNE(n_components=2, random_state=42).fit_transform(all_embeds)

# Plot
plt.figure(figsize=(6,6))
plt.scatter(X_2d[:,0], X_2d[:,1], c=all_labels, cmap="coolwarm", alpha=0.7)
plt.colorbar(label="Class")
plt.title("t-SNE of Validation Embeddings")
plt.show()